<a href="https://colab.research.google.com/github/pollinations/hive/blob/main/notebooks/2%20Text-To-Video/2%20LIHQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Image](https://github.com/johnGettings/LIHQ/raw/master/docs/demo_gif.gif)

Long-Inference, High Quality Synthetic Speaker. Probably only works with Colab Pro at the moment.

In [ ]:
text = 'The carrot has mystery.'  #@param {type: "string"}

output_path = "/content"

# **1) SET UP**

In [ ]:
#Git repos

#LIHQ
!git clone https://github.com/johnGettings/LIHQ

%cd LIHQ
#Tortoise
!git clone https://github.com/neonbjb/tortoise-tts.git
#FOMM
!git clone https://github.com/AliaksandrSiarohin/first-order-model
#Wav2Lip
!git clone https://github.com/Rudrabha/Wav2Lip.git
#GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
#QVI
!git clone https://github.com/xuxy09/QVI
#MODNet
!git clone https://github.com/ZHKKKe/MODNet


###LIHQ
!pip install -r requirements.txt

###Tortoise
%mv '/content/LIHQ/tortoise-tts' '/content/LIHQ/tortoise_tts'
%cd /content/LIHQ/tortoise_tts
!python3 setup.py install

###FOMM
%mv '/content/LIHQ/first-order-model' '/content/LIHQ/first_order_model'
%cd /content/LIHQ/first_order_model
#getting model weights
!gdown --id 1DbjXD2nS3jlyCWoJu2HGcLZZjhLC9a2J

###Wav2Lip
%cd /content/LIHQ/Wav2Lip
#Downloading model weights
!gdown --id 1eAtM-Ck5RMyMMZoQuoQfYZRU5vDDwBpK -O './checkpoints/wav2lip_gan.pth'
!gdown --id 1eAtM-Ck5RMyMMZoQuoQfYZRU5vDDwBpK -O './checkpoints/wav2lip.pth'
!wget -c "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "./face_detection/detection/sfd/s3fd.pth"

#GFPGAN
%cd /content/LIHQ/GFPGAN
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
!wget -c https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

###QVI
%cd /content/LIHQ/QVI
!wget -c https://www.dropbox.com/s/5auirpk1tijxo1v/model.pt?dl=0 -O './model.pt' 
!wget -c https://www.dropbox.com/s/afsqhlzu0rarpmf/pwc-checkpoint.pt?dl=0 -O './utils/pwc-checkpoint.pt'
%rm './demo.py'
!gdown --id 1xl7ziToar2ZDNpToOOKc8fugw1xeQ6fp -O './demo.py'

###MODNET
%cd /content/LIHQ/MODNet
!gdown --id 1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz -O pretrained/modnet_photographic_portrait_matting.ckpt

%cd /content/LIHQ

# **Example 1: Simple Speaker with StyleGAN2 and TorToiSe (Recommended Use)**

**Generating audio from TorToiSe** (Or you can always put your own audio in the audio folders)

In [ ]:
# Run this once to initialize. Takes about a minute
%cd /content/LIHQ/tortoise_tts
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

from LIHQ.procedures.tortoise_scripts import tortoise_run, tortoise_combo_run

tts = TextToSpeech()

In [ ]:
# Generating a random voice to speak the sentence below. When you generate one you like, save it using the cell below
voice = 'random'
preset = "high_quality" #Options: {"ultra_fast", "fast", "standard", "high_quality"}.

gen = tortoise_run(tts, text, voice, preset)
#IPython.display.Audio(gen.squeeze(0).cpu(), rate = 24000)

In [ ]:
# Saving audio from above. Just change the foldername & filename as needed
torchaudio.save('/content/LIHQ/input/audio/Folder1/2.wav', gen.squeeze(0).cpu(), 24000)

**Silence Padding**

In [ ]:
# Adding half a second of silence before audio, because FOMM works better when first frame is a closed mouth.
# Also helps with abrupt start/ stop if you want to add silence after as well.
from pydub import AudioSegment

silence_duration = 500  # Set duration in milliseconds. 1000 is 1 second
save_path = '/content/LIHQ/input/audio/Folder1/1.wav'

silent_segment = AudioSegment.silent(duration=silence_duration)  
silent_segment.export(save_path, format="wav")

**Running main script**

In [ ]:
!ls -l /content/LIHQ/input/audio/Folder1/
%cd /content/LIHQ
from runLIHQ import run

run(face='/content/LIHQ/input/face/examples/2372448.png')

encoding_options = "-c:v libx264 -crf 20 -preset slow -vf format=yuv420p -c:a aac -movflags +faststart"
!ffmpeg -i /content/LIHQ/output/finalVidsOut/* {encoding_options} {output_path}/video.mp4


# **Example 2: Multiple Speakers, Replacing Background**

**Generating audio from TorToiSe**